In [1]:
###TODO:
# cal comparison does not account for pages i think.


#imports
from tkinter import *
import os
import re
import time
import datetime

import telnetlib
import getpass 
import sys
import time
import serial
import threading #potentially replaced by multiprocessing
from multiprocessing.pool import ThreadPool

import pandas as pd
import numpy as np

from pdfrw import PdfReader
import matplotlib.pyplot as plt
import PyPDF2
from tkinter.filedialog import askopenfilename
os.environ.setdefault('MAGICK_HOME', '/usr/local/Cellar/imagemagick@6/6.9.9-34/')
from wand.image import Image
from PIL import Image as PI
import pyocr
import pyocr.builders
import io


#global defines
nport_ip = "10.136.1.99"
ref_port = "4001"



In [9]:
#generate "main" gui window in tkinter
def onclick(): 
        pass

#updates the main status label with given text.  Could be accomplished by just calling "main_status.config" from within the main function...
def update_txt(next_instruction):
    main_status.config(text=next_instruction)   
    
#def connex(port):
    #print(port)
    
win = Tk()

win.title('SBE CTD Prequalification')
win.geometry('1000x600') # Size 200, 200

text = Text(win, height=2)

#labels
main_status = Label(win, text="Waiting for device selection")
calsheet_status_label = Label(win, text="")
sample_status_label = Label(win, text="")

#Buttons
port2 = Button(win, text="4002 - 485", command=lambda: instrument_connection("4002"))
port3 = Button(win, text="4003 - 232", command=lambda: instrument_connection("4003"))
port4 = Button(win, text="4004", command=lambda: instrument_connection("4004"))
#exit_button = Button(win, text="Exit", command=win.destroy())

text.insert(INSERT, "Please connect the RMA device and specify the port used")

text.pack()
port2.pack()
port3.pack()
port4.pack()
main_status.pack()
calsheet_status_label.pack()
sample_status_label.pack()
#exit_button.pack()

#btn = Button(win, text='Hello', command=hello)
#btn.pack(expand=YES, fill=BOTH)

mainloop()


((['TA0', 'TA1', 'TA2', 'TA3', 'G', 'H', 'I', 'J', 'CPCOR', 'CTCOR', 'WBOTC', 'reference pressure'], ['4.261456e-05', '2.731360e-04', '-2.206636e-06', '1.490857e-07', '-9.994807e-01', '1.566288e-01', '-3.317297e-04', '4.905268e-05', '-9.570000e-08', '3.250000e-06', '1.185617e-06', '2007.0 decibars']), [None, None, None, None, '—1.0l7709e+000', '1.328397e—001', '—l.413619e—004', None, '~9.5700e—008', '3.2500e—006', None, None])
stringvar:  PY_VAR36
stringvar:  PY_VAR37
stringvar:  PY_VAR38
stringvar:  PY_VAR39
stringvar:  PY_VAR40
stringvar:  PY_VAR41
stringvar:  PY_VAR42
stringvar:  PY_VAR43
stringvar:  PY_VAR44
stringvar:  PY_VAR45
stringvar:  PY_VAR46
stringvar:  PY_VAR47
stringvar:  PY_VAR36
stringvar:  PY_VAR37
stringvar:  PY_VAR38
stringvar:  PY_VAR39
stringvar:  PY_VAR40
stringvar:  PY_VAR41
stringvar:  PY_VAR42
stringvar:  PY_VAR43
stringvar:  PY_VAR44
stringvar:  PY_VAR45
stringvar:  PY_VAR46
stringvar:  PY_VAR47
stringvar:  PY_VAR36
stringvar:  PY_VAR37
stringvar:  PY_VAR38
st

In [3]:
# Instrument Connection
#module to:
# 1. Prompt user to select NPORT port RMA instrument is connected to (maybe goes in main gui window?)
# 2. Create a telnet connection to both instruments
# 3. Send wakeup to both instruments
# 4. Alert user that connection successful

#entry point to module
def instrument_connection(rma_port):
    global nport_ip
    global ref_port
    device_dict = {
        "4002" : "id?\r\n",
        "4003" : "\r\n"
    }
    try:
        ref_telnet = telnetlib.Telnet(nport_ip, ref_port) #open connection
        rma_telnet = telnetlib.Telnet(nport_ip, rma_port)
        update_txt("Connecting, please wait...")
        time.sleep(1)
        ref_telnet.write(device_dict['4003'].encode("ascii"))  #wakeup both instruments
        rma_telnet.write((device_dict[rma_port]).encode("ascii"))
        time.sleep(1)
        rma_telnet.write((device_dict[rma_port]).encode("ascii"))
        time.sleep(1)
        rma_ok = rma_telnet.read_very_eager()
        ref_ok = ref_telnet.read_very_eager()
        ref_telnet.close() #close reference connection, not done with rma yet
        rma_telnet.close()
    except:
        print("bricked it")
        ref_telnet.close()
        rma_telnet.close()
    finally:
        ref_telnet.close()
        rma_telnet.close()
    update_txt("Connection Successful")
    calsheet_status_label.config(text="Collecting RMA calibration data...")    
    compcals = compare_cals_entrypoint(rma_port) #pass the telnet connection to compare_cals function
    #samples = collect_samples(rma_port)
    print(compcals)
    gen_report(compcals)    

In [4]:
# Compare Samples
#module to:
# 1. Collect 10 samples (1/min) from each instrument
# 2. Compare samples at like timestamps(should be +/- 1s)
# 3. Format data and present to user, advise pass/fail, generate comparison column as well

def connect_ref(s):
    tn = s
    tn.write(("\r\n").encode('ascii'))
    time.sleep(2)
    sample_time = datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S:%f')
    tn.write(("TS\r\n").encode('ascii'))
    time.sleep(60)
    data = str(tn.read_very_eager())
    data = sample_time + ', '+ data
    #print(data)
    tn.close()
    return data
    
def connect_rma(s):
    tn = s
    tn.write(("id?\r\n").encode('ascii'))
    time.sleep(1)
    sample_time = datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S:%f')
    tn.write(("#03TS\r\n").encode('ascii'))
    time.sleep(60)
    data = str(tn.read_very_eager())
    data = sample_time + ', '+ data
    tn.close()
    return data

def clean_sample(dirty_sample):
    trantab = str.maketrans('', '', 'b\'\\rn<Executed/>TS=i Bpls')
    translate = dirty_sample.translate(trantab)
    return translate
    
def sample_todf(ref_sample, rma_sample, rma_port):
    ref_sample = ref_sample.split(',')[:4]
    rma = rma_sample.split(',')
    if rma_port == "4002": #correct this for ins type - potentially need a check for pressure
        rma_sample = [rma[0], rma[3], rma[4], rma[5]]
    if rma_port == "4003":     #correct this for ins type
        rma_sample = rma[:3]
    df = pd.DataFrame(data=[ref_sample, rma_sample], columns=['datetime', 'temperature', 'conductivity', 'pressure'])
    return df
      
def collect_samples(rma_port):
    global nport_ip
    
    pool = ThreadPool(processes=2)
    rma_tn = telnetlib.Telnet(nport_ip, "4002", 5)
    ref_tn = telnetlib.Telnet(nport_ip, "4001", 5)    
    procRMA = pool.apply_async(connect_rma, args=(rma_tn,))
    procREF = pool.apply_async(connect_ref, args=(ref_tn,))
    rma_sample = procRMA.get()
    ref_sample = procREF.get()
    rma_sample = clean_sample(rma_sample)
    ref_sample = clean_sample(ref_sample)
    return sample_todf(ref_sample, rma_sample, rma_port)
    #print(rma_sample +'\n'+ref_sample)
#     threadRMA = threading.Thread(target=connect_rma, args=(rma_tn,),).start()
#     threadREF = threading.Thread(target=connect_ref, args=(ref_tn,),).start()
    
    

In [8]:
# Compare Cals
#module to:
# 1. Prompt user for calibration sheet location
# 2. Collect cals from instrument(dcal, #iidc)
# 3. Compare Results
# 4. Display side by side results AND suggest an action(pass/fail)
# 5. If "pass" then continue after ~30seconds if "fail" pause and wait for user?\
def conv_to_float(x):  #lambda float conversion
    try:
        return float(x)
    except:
        return 0

def compare_cals_entrypoint(rma_port):
    instr_test = 'SBE37SI-RS485 V 3.2  5679\r\ntemperature:  25-Mar-17\r\n    TA0 = 4.261456e-05\r\n    TA1 = 2.731360e-04\r\n    TA2 = -2.206636e-06\r\n    TA3 = 1.490857e-07\r\nconductivity:  25-Mar-17\r\n    G = -9.994807e-01\r\n    H = 1.566288e-01\r\n    I = -3.317297e-04\r\n    J = 4.905268e-05\r\n    CPCOR = -9.570000e-08\r\n    CTCOR = 3.250000e-06\r\n    WBOTC = 1.185617e-06\r\nreference pressure = 2007.0 decibars\r\n<Executed/>\r\n'
    sheet_test = open("Output.txt", 'r').read()
    if rma_port == "4002":
        ins_caldata = get_cals_37()
    if rma_port == "4003":
        ins_caldata = get_cals_19()
    sheet_caldata = get_sheet_cals()
    cal_values = compare_cals(ins_caldata, sheet_caldata)
    #df = pd.DataFrame(data = compare_cals(ins_caldata, sheet_caldata), columns=['Tag', 'Ins Val', 'Sheet Val'])#columns=['Value', 'Instrument', 'Sheet'])
#     try:
#         df['Ins Val'] = df['Ins Val'].apply(lambda x: x.replace('~','-').replace('€', 'e'))
#         df['Sheet Val'] = df['Sheet Val'].apply(lambda x: x.replace('~','-').replace('€', 'e'))
#     except: 
#         pass
    #df['Instrument'] = df['Instrument'].apply(lambda x: conv_to_float(x))
    #df['Sheet'] = df['Sheet'].apply(lambda x: conv_to_float(x))   
    #df['Difference'] = abs((df['Instrument']) - (df['Sheet']))
    #update_cals(df)  ##THOROUGH TEST BEFORE FIRING THIS ONE
    return cal_values
    
def get_cals_37():
    global nport_ip
    rma_telnet = telnetlib.Telnet(nport_ip, "4002")
    rma_telnet.write("id?\r\n".encode("ascii"))
    time.sleep(1)
    rma_telnet.write("id?\r\n".encode("ascii"))
    time.sleep(1)
    sbe_id = str(rma_telnet.read_very_eager())[5:7]
    rma_telnet.write(("#"+sbe_id+"DC\r\n").encode("ascii"))
    time.sleep(1)
    #print(rma_telnet.read_very_eager()) #cal data to be parsed
    caldata = rma_telnet.read_very_eager()
    rma_telnet.close()
    return caldata #.decode('unicode-escape')
def get_cals_19():
    rma_telnet = telnetlib.Telnet(nport_ip, "4002")
    rma_telnet.write("DCal\r\n".encode("ascii"))
    time.sleep(1)
    rma_telnet.write("DCal\r\n".encode("ascii"))
    time.sleep(1)
    #print(rma_telnet.read_very_eager()) #cal data to be parsed
    caldata = rma_telnet.read_very_eager()
    rma_telnet.close()
    return str(caldata)

#this function doesn't actually need to exist, but helps with clarity i think?
def get_sheet_cals():
    filepath = request_filepath()
    return read_file(filepath)
    

#get user input filepath(browse files)    
def request_filepath():
    file_select = Tk()
    file_select.withdraw()
    filename=askopenfilename()
    file_select.update()
    file_select.destroy()
    return filename

#OCR on file, parse out to text
def read_file(filepath):
    tool = pyocr.get_available_tools()[0]
    lang = tool.get_available_languages()[0]
    req_image = []
    final_text = []
    image_pdf = Image(filename=filepath, resolution=300)
    image_jpeg = image_pdf.convert('jpeg')
    for img in image_jpeg.sequence:
        img_page = Image(image=img)
        req_image.append(img_page.make_blob('jpeg'))
    for img in req_image: 
        txt = tool.image_to_string(
            PI.open(io.BytesIO(img)),
            lang=lang,
            builder=pyocr.builders.TextBuilder()
        )
        final_text.append(txt)
    merge_text = ''.join(final_text)
    return merge_text

def split_sheet(sheet):
    try:
        m = sheet[0].split(' ')[0]
        return m
    except:
        pass

#compare cal file and instruments put into 2 column dataframe 
#no comparison yet just collects and columnizes

#pass strings for cal sheet and instrument cals and it will compare all values and generate a comparison report
def compare_cals(instr, sheet):
    i_s = instr
    s_s = sheet
    comparison_array = []
    #print(s_s)
    instrument = parse_cal_tags(i_s.decode('unicode-escape'))
    sheet_return = parse_sheets(s_s, instrument)     
#     for i in range(len(instrument[0])):
#         try:
#             comparison_array.append((instrument[0][i], instrument[1][i], sheet_return[i]))
#         except:
#             print(instrument[0][i], instrument[1][i], sheet_return[i])
#     #print(comparison_array)
#     return comparison_array
        
#     for i in range(len(instrument[0])):
#         try:
#             comparison_array.append(instrument[0][i], instrument[1][i], sheet_return[i])
#         except:
#             pass
#     print(comparison_array)
    return (instrument, sheet_return)
    
def find_value(tag, input_file):
    m = re.findall(tag, input_file)
    if m:
        return m
    else:
        return "0"

def split_sheet(sheet):
    try:
        m = sheet[0].split(' ')[0]
        return m
    except:
        pass

def parse_sheets(s_s, instr):    
    sheet_return = []
    #instrument = parse_cal_tags(instr)
    instrument = instr
    for i in range(len(instrument[0])):
        tags = instrument[0]
        sheet_regex = r"(?i)(?<=\s" + instrument[0][i] + " = )(.*)(?=\s)"
        m = re.findall(sheet_regex, s_s)
        sheet_return.append(split_sheet(m))
    return sheet_return
    
                   
def parse_cal_tags(cal_string):
    m = re.findall(r"(?<=\s)(.*)(?=\s)", cal_string) 
    calibration_tags = []
    calibration_rows = []
    for x in m:
        entry = x.split('=')
        try:
            if len(entry) >= 2:
                calibration_tags.append(entry[0].strip())
                calibration_rows.append(entry[1].strip())
        except:
            pass
    return (calibration_tags, calibration_rows)
                       

In [61]:
# Cleanup
# 1. Close telnet connections
# 2. Save any relevant tables/data to files(not sure if any exists)

#print(find_value(r'[^=]*', "A0 = 1.2345e-003 A1 = 67537653"))
#print(find_value(r"(?<=[=])[^\\\n]*", "A0 = 1.2345e-002  \n A1 = 2.2356"))
# float(find_value(r"(?<=[=])[^\\\n]*", "A0 = 1.2345e-002"))

string = "A0 = 1.2345e-002  \n cptocor1 = 2.2356"
#tag = r"(?:(?!=).)*"
tag = r".+?[=]"
m = re.findall(tag, string, re.M)



' cptocor1'

In [12]:
a = "a1 = 131452345"
b = a[:2]
print(b)

a1


In [5]:
#put samples into DF - perform comparison  
rma_sample = '2018030814:25:04:552767,0303,03705679,22.0795,-0.00001,1986.440,1522.037,6.6957,08Ma2018,22:31:09'
ref_sample = '2018030814:25:05:554553,21.8642,0.03970,-0.168,17.315,0.626569,0.2043,1488.172'

##should get passed the type of instrument not the port - port type is irrelevant at this stage
def sample_todf(ref_sample, rma_sample, rma_port):
    ref_sample = ref_sample.split(',')[:4]
    rma = rma_sample.split(',')
    if rma_port == "4002": #correct this for ins type - potentially need a check for pressure
        rma_sample = [rma[0], rma[3], rma[4], rma[5]]
    if rma_port == "4003":     #correct this for ins type
        rma_sample = rma[:3]
    df = pd.DataFrame(data=[ref_sample, rma_sample], columns=['datetime', 'temperature', 'conductivity', 'pressure'])
    return df
    
#print (sample_todf(ref_sample, rma_sample))    
sample_todf(ref_sample, rma_sample, "4002").head()

,datetime,temperature,conductivity,pressure
0,2018030814:25:05:554553,21.8642,0.03970,-0.168
1,2018030814:25:04:552767,22.0795,-0.00001,1986.440


In [6]:
#report generation window
# def gen_report(cals, samples):
#     report=Tk()
#     report.geometry("1000x600")
#     report.title("Report")
    
#     #title for cal data
#     cal_label = Label(report, text="Calibration Comparison", height=1, width=50)
#     cal_label.grid()
#     #title for sample data
#     sample_label = Label(report, text="Sample Comparison", height=1)
#     sample_label.grid(row=0, column=1)
    
#     #samples data
#     sample_comparison = Text(report, bg='blue', height=5)
#     sample_comparison.grid(row=1, column=1, sticky='NW')
#     sample_comparison.insert(INSERT, samples)
    
#     #summary of results(written)
#     text_summary = Text(report)
#     text_summary.grid(row=2, column=1, sticky = 'NW')
#     text_summary.insert(INSERT, "PASS OR FAILLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLL")

#     #calibration data
#     data = Text(report, bg='red', height=20)
#     data.grid(row=1, column=0, rowspan=2, sticky='NW')
#     data.insert(INSERT, cals)
#     length = cals.shape[0]
#     container = [None] * length
    
#     report = Tk()
#     report.geometry("1000x600")
#     report.title("Report")
    
#     cal_label = Label(report, text="Calibration Comparison", height=1, width=50)
#     cal_label.grid()
    
#     data = Text(report, bg='red', height=20)
#     data.grid(row=1, column=0, rowspan=2, sticky='NW')
#     data.insert(INSERT, cals)
        
#     for i in range(length):
#         container[i] = Label(report, text = i)
        
def gen_report(cals):
    def stringvar_list(lst):
        vals = [str(i) for i in lst]
        ret_lst = [None]*len(lst)
        for i in range(len(lst)):
            ret_lst[i] = StringVar()
            ret_lst[i].set(vals[i])
        return ret_lst
    def update_cal_var():
        for i in range(length):
            try:
                sheet_values[i].set(update_val[i].get())
                print("stringvar: ", update_val[i])
            except Exception as e:
                print (e)
                
#     report = Tk()
#     report.geometry("1000x600")
#     report.title("Report")
    for widget in frame.winfo_children():
        widget.destroy()
    
    instr_data = cals[0]
    sheet_data = cals[1]
    
    
    sheet_data = ['None' if v is None else v for v in sheet_data]

    instr_tags = stringvar_list(instr_data[0])
    instr_values = stringvar_list(instr_data[1])
    sheet_values = stringvar_list(sheet_data)
    
        
    length = len(instr_data[0])
    tags = [None] * length
    ins_val = [None] * length
    she_val = [None] * length
    update_entry = [None] * length
    update_val = [None]*length
    
    cal_label = Label(report, text="Calibration Comparison", height=1, width=20)
    cal_label.grid(sticky='NW', columnspan=3)
          
    for i in range(length):
        tags[i] = Text(report, height=1, width=15)
        tags[i].insert(INSERT, instr_tags[i].get())
        tags[i].grid(column=0, row=i+1)
        ins_val[i] = Text(report, height=1, width=15)
        ins_val[i].insert(INSERT, instr_values[i].get())
        ins_val[i].grid(column=1, row=i+1, sticky='NW')
        she_val[i] = Label(report, height=1, width=15, text=sheet_values[i].get() , textvariable=sheet_values[i])
        she_val[i].grid(column=2, row=i+1, sticky='NW', padx = 10)
        update_val[i] = StringVar()
        update_entry[i] = Entry(report ,textvariable = update_val[i])
        update_entry[i].insert(END, sheet_values[i].get())
        update_entry[i].grid(column=3, row=i+1, sticky='NW')
    
    update_button = Button(report, text='Update Values', command=update_cal_var)
    update_button.grid(row=length+1, columnspan=3)
        
    mainloop()
    
    
    
    

In [ ]:
#update cal values on instrument if they are out of line with cal sheet
def update_cals(cals):
    commands_sbe37 = ['TCalDate=', 'TA0=', 'TA1=','TA2=','TA3=', 'CCalDate=', 'CG=', 'CH=', 'CI=', 'CJ=', 'WBOTC=', 'CTCor=', 'CPCor=', 'PCalDate=', 'PA0=','PA1=','PA2=','PTCA0=','PTCA1=','PTCA2=','PTCB0=','PTCB1=','PTCB2=', 'PTempA0=', 'PTempA1=', 'PTempA2=', 'POffset=']
    commands_sbe19 = ['TCalDate=', 'TA0=', 'TA1=','TA2=','TA3=', 'TOffset=', 'CCalDate=', 'CG=', 'CH=', 'CI=', 'CJ=', 'CTCor=', 'CPCor=', 'CSlope', 'PCalDate=', 'PRange=', 'POffset=', 'PA0=','PA1=','PA2=', 'PTCB0=','PTCB1=','PTCB2=', 'PTempA0=', 'PTempA1=', 'PTempA2=']
    
    #
    

In [11]:
rma_tn.close()
ref_tn.close()

NameError: name 'rma_tn' is not defined

In [13]:
#setup for doing manual sampling
rma_tn = telnetlib.Telnet(nport_ip, "4002", 5)
#ref_tn = telnetlib.Telnet(nport_ip, "4001", 5)  
#rma_tn.write(("#03OutputFormat=1\r\n").encode('ascii'))
#time.sleep(3)
data = []
rma_tn.write(("#03DC\r\n").encode('ascii'))
time.sleep(2)
# rma_tn.write(("#03TS\r\n").encode('ascii'))
# time.sleep(50)
data = rma_tn.read_very_eager()
rma_tn.close()
#ref_tn.close()
print(data)

SBE37SI-RS485 V 3.2  5679
temperature:  25-Mar-17
    TA0 = 4.261456e-05
    TA1 = 2.731360e-04
    TA2 = -2.206636e-06
    TA3 = 1.490857e-07
conductivity:  25-Mar-17
    G = -9.994807e-01
    H = 1.566288e-01
    I = -3.317297e-04
    J = 4.905268e-05
    CPCOR = -9.570000e-08
    CTCOR = 3.250000e-06
    WBOTC = 1.185617e-06
reference pressure = 2007.0 decibars
<Executed/>



In [28]:
# rma_tn.close()
# ref_tn.close()
m = re.findall(b"(?<=\s)(.*)(?=\s)", data) 
parse_cal_tags(data.decode('unicode-escape'))

(['TA0',
  'TA1',
  'TA2',
  'TA3',
  'G',
  'H',
  'I',
  'J',
  'CPCOR',
  'CTCOR',
  'WBOTC',
  'reference pressure'],
 ['4.261456e-05',
  '2.731360e-04',
  '-2.206636e-06',
  '1.490857e-07',
  '-9.994807e-01',
  '1.566288e-01',
  '-3.317297e-04',
  '4.905268e-05',
  '-9.570000e-08',
  '3.250000e-06',
  '1.185617e-06',
  '2007.0 decibars'])

In [83]:
#instrument
#r"(?<=CPcor = )(.*)(?=\s)"
def split_sheet(sheet):
    try:
        m = sheet[0].split(' ')[0]
        return m
    except:
        pass

sheet_return = []
for i in range(len(instrument[0])):
    tags = instrument[0]
    sheet_regex = r"(?i)(?<=\s" + instrument[0][i] + " = )(.*)(?=\s)"
    m = re.findall(sheet_regex, s_s)
    sheet_return.append(split_sheet(m))
    

[None,
 None,
 None,
 None,
 '—1.0l7709e+000',
 '1.328397e—001',
 '—l.413619e—004',
 None,
 '~9.5700e—008',
 '3.2500e—006',
 None,
 None]

In [31]:
global nport_ip
rma_telnet = telnetlib.Telnet(nport_ip, "4002")
rma_telnet.write("id?\r\n".encode("ascii"))
time.sleep(1)
rma_telnet.write("id?\r\n".encode("ascii"))
time.sleep(1)
sbe_id = str(rma_telnet.read_very_eager())[5:7]
rma_telnet.write(("#"+sbe_id+"DC\r\n").encode("ascii"))
time.sleep(1)
#print(rma_telnet.read_very_eager()) #cal data to be parsed
caldata = rma_telnet.read_very_eager()
rma_telnet.close()

caldata

b'SBE37SI-RS485 V 3.2  5679\r\ntemperature:  25-Mar-17\r\n    TA0 = 4.261456e-05\r\n    TA1 = 2.731360e-04\r\n    TA2 = -2.206636e-06\r\n    TA3 = 1.490857e-07\r\nconductivity:  25-Mar-17\r\n    G = -9.994807e-01\r\n    H = 1.566288e-01\r\n    I = -3.317297e-04\r\n    J = 4.905268e-05\r\n    CPCOR = -9.570000e-08\r\n    CTCOR = 3.250000e-06\r\n    WBOTC = 1.185617e-06\r\nreference pressure = 2007.0 decibars\r\n<Executed/>\r\n'